## Gray-Scott Model

Represents the reaction and diffusion of two generic chemical species U and V.  The concentration of each in space is represented by u and v.

* Each chemical diffuses through space at its own rate
* U is added at a const feed rate into the system
* Two units of species V can turn a unit of U into V: $$2V + U \rightarrow 3V$$
* Const kill rate removing V

Represented by PDE for the concentrations of $u(x, y, t)$ and $v(x, y, t)$:

$$\frac{\partial u}{\partial t} = D_u \nabla ^2 u - uv^2 + F(1 - u)$$
$$\frac{\partial v}{\partial t} = D_v \nabla ^2 v + uv^2 - (F + k)v$$

$\nabla ^2$ is the Laplacian:

$$\nabla ^2 u = \frac{\partial ^2 u}{\partial x^2} + \frac{\partial ^2 u}{\partial y^2}$$

$D_u \nabla ^2 u$ and $D_v \nabla ^2 v$ correspond to the spatial diffusion of the concentrations.  $D_u$ and $D_v$ are the rates of diffusion.

$uv^2$ corresponds to the reaction which requires one unit of U and two of V.

$F(1 - u)$ is the feed rate and $-(F + k)v$ is the kill rate.  The feed rate replenishes U and the kill rate diminishes V.

### Problem setup

* Discretize reaction-diffusion using FTCS and assume $\Delta x = \Delta y = \delta$
* For the timestep set $\Delta t = \frac{9}{40} \frac{\delta ^2}{max(D_u, D_v)}$
* Use zero Neumann BC on all sides of the domain
* Use given ICs
    * Grid of pointswith dimension 192 by 192 points
    * Domain is 5m by 5m
    * Final time is 8000s

In [ ]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
n = 192

Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 #Bacteria 1

dh = 5 / (n - 1)

T = 8000

dt = .9 * dh ** 2 / (40 * max(Du, Dv))

nt = int(T / dt)

In [ ]:
uvinitial = numpy.load('./uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']

In [ ]:
fig = pyplot.figure(figsize = (8, 5))
pyplot.subplot(121)
pyplot.imshow(U, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);
pyplot.subplot(122)
pyplot.imshow(V, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);

### Discretization with forward time and central space

$$\Delta x = \Delta y = \delta$$
$$\delta = dh$$

$$\frac{u_{i,j} ^{n+1} - u_{i,j} ^n}{\Delta t} = D_u \frac{\left( u_{i-1,j} ^n - 2u_{i,j} ^n + u_{i+1,j} ^n \right) }{dh^2} + D_u \frac{ \left( u_{i,j-1} ^n - 2u_{i,j} ^n + u_{i,j+1} ^n \right) }{dh^2} - \left( u_{i,j} ^n \circ v_{i,j} ^n \circ v_{i,j} ^n \right) + F(1 - u_{i,j}^n)$$

$$\frac{v_{i,j} ^{n+1} - v_{i,j} ^n}{\Delta t} = D_v \frac{\left( v_{i-1,j} ^n - 2v_{i,j} ^n + v_{i+1,j} ^n \right) }{dh^2} + D_v \frac{ \left( v_{i,j-1} ^n - 2v_{i,j} ^n + v_{i,j+1} ^n \right) }{dh^2} + \left( u_{i,j} ^n \circ v_{i,j} ^n \circ v_{i,j} ^n \right) - (F + k) \circ (v_{i,j} ^n)$$

Put everything except $u_{i,j} ^{n+1}$ and $v_{i,j} ^{n+1}$ on the right side

$$u_{i,j} ^{n+1} = u_{i,j} ^n + \Delta t \left[ D_u \frac{\left( u_{i-1,j} ^n - 2u_{i,j} ^n + u_{i+1,j} ^n \right) }{dh^2} + D_u \frac{ \left( u_{i,j-1} ^n - 2u_{i,j} ^n + u_{i,j+1} ^n \right) }{dh^2} - \left( u_{i,j} ^n \circ v_{i,j} ^n \circ v_{i,j} ^n \right) + F(1 - u_{i,j}^n) \right]$$

$$v_{i,j} ^{n+1} = v_{i,j} ^n + \Delta t \left[ D_v \frac{\left( v_{i-1,j} ^n - 2v_{i,j} ^n + v_{i+1,j} ^n \right) }{dh^2} + D_v \frac{ \left( v_{i,j-1} ^n - 2v_{i,j} ^n + v_{i,j+1} ^n \right) }{dh^2} + \left( u_{i,j} ^n \circ v_{i,j} ^n \circ v_{i,j} ^n \right) - (F + k) \circ (v_{i,j} ^n) \right]$$

In [ ]:
#def ftcs(U, V, dt, dh, Du, Dv, F, k):
    
for n in range(nt):
        
    un = U.copy()
    vn = V.copy()
        
    U[1:-1, 1:-1] = un[1:-1, 1:-1] + dt * ((Du * (un[:-2, 1:-1] - 2 * un[1:-1, 1:-1] + un[2:, 1:-1]) / dh ** 2) + (Du * (un[1:-1, :-2] - 2 * un[1:-1, 1:-1] + un[1:-1, 2:]) / dh ** 2) - (un[1:-1, 1:-1] * vn[1:-1, 1:-1] * vn[1:-1, 1:-1]) + F * (1 - un[1:-1, 1:-1]))
        
    V[1:-1, 1:-1] = vn[1:-1, 1:-1] + dt * ((Dv * (vn[:-2, 1:-1] - 2 * vn[1:-1, 1:-1] + vn[2:, 1:-1]) / dh ** 2) + (Dv * (vn[1:-1, :-2] - 2 * vn[1:-1, 1:-1] + vn[1:-1, 2:]) / dh ** 2) + (un[1:-1, 1:-1] * vn[1:-1, 1:-1] * vn[1:-1, 1:-1]) - (F + k) * vn[1:-1, 1:-1])
        
    #Neumann BC
        
    U[0, :] = U[1, :]
    U[:, 0] = U[:, 1]
    U[-1, :] = U[-2, :]
    U[:, -1] = U[:, -2]
        
    V[0, :] = V[1, :]
    V[:, 0] = V[:, 1]
    V[-1, :] = V[-2, :]
    V[:, -1] = V[:, -2]
        
    #return U, V

In [ ]:
x = numpy.linspace(0, 5, n)

y = numpy.linspace(0, 5, n)

mx, my = numpy.meshgrid(x, y)

#U, V = ftcs(U, V, dt, dh, Du, Dv, F, k)

In [ ]:
#pyplot.figure(figsize = (8,5))
#pyplot.subplot(121)
#pyplot.contourf(mx, my, U);
#pyplot.subplot(122)
#pyplot.contourf(mx, my, V);

In [ ]:
pyplot.figure(figsize = (8, 5))
pyplot.subplot(121)
pyplot.imshow(U, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);
pyplot.subplot(122)
pyplot.imshow(V, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);